#### Dependencies

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import librosa
import time
import multiprocessing
from dotenv import load_dotenv


from essentia.standard import (
    MonoLoader,
    Danceability,
    Spectrum,
    FrameCutter,
    Loudness,
    RhythmExtractor2013,
    KeyExtractor,
    Energy,
    TonalExtractor,
    Inharmonicity,
    MFCC,
    OnsetRate,
    SpectralCentroidTime,
    DynamicComplexity,
    SpectralPeaks,
    NoveltyCurve,
    Spectrum,
    FrameGenerator,
    Windowing,
    MelBands,
    BeatsLoudness,
    Beatogram,
    Meter,
    HumDetector,
)

In [5]:
with open('data/mtg_jamendo_moodtheme-discogs-effnet-1.json', 'r') as jamendo_file:
    jamendo_metadata = json.load(jamendo_file)
jamendo_classes = jamendo_metadata['classes']

with open('data/mtg_jamendo_instrument-discogs-effnet-1.json', 'r') as jamendo_file:
    jamendo_instrument_metadata = json.load(jamendo_file)
jamendo_instrument_classes = jamendo_instrument_metadata['classes']

#### Global Constants

In [6]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = 16

#### Data

In [7]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Feature Extraction Functions

In [8]:
def create_spectrogram_image(spectrogram_db, sample_rate):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=11025)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel-Spectrogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [9]:
def convert_mp3_to_spectrogram(audio_path, create_image=False):
    mp3, sample_rate = librosa.load(audio_path, sr=22050)
    spectrogram = librosa.feature.melspectrogram(y=mp3, sr=sample_rate, n_mels=128, fmax=11025)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    if create_image:
        create_spectrogram_image(spectrogram_db, sample_rate)

    return spectrogram_db, sample_rate

In [10]:
def get_mel_bands(audio):
    spectrum = Spectrum()
    frame_generator = FrameGenerator(audio, frameSize=2048, hopSize=1024)
    window = Windowing(type='hann')

    mel_bands = MelBands(numberBands=40)
    mel_band_energies = []

    for frame in frame_generator:
        spec = spectrum(window(frame))
        mel_band_energies.append(mel_bands(spec))

    mel_band_energies = np.array(mel_band_energies)
    return mel_band_energies

In [11]:
def run_essentia_algorithms(audio44k):
    _, mfcc_coeffs = MFCC(inputSize=len(audio44k))(audio44k)
    danceability_score = Danceability()(audio44k)
    loudness_score = Loudness()(audio44k)
    bpm, beat_positions, _, _, _ = RhythmExtractor2013(method="multifeature")(audio44k)
    key, scale, _ = KeyExtractor()(audio44k)
    energy_score = Energy()(audio44k)

    ### Chord Significances
    _, _, _, _, chords, _, _, _, _, _, _, _ = TonalExtractor()(audio44k)
    unique_chords, counts = np.unique(chords, return_counts=True)
    chords_significance = {chord: significance for (chord, significance) in zip(unique_chords, counts)}
    
    ### Inharmonicity
    frames = []
    frameCutter = FrameCutter()
    while True:
        frame = frameCutter(audio44k)
        if not len(frame):
            break
        frames.append(frame)
        
    spectrum_magnitudes = []
    for frame in frames:
        spectrum_magnitudes_frame = Spectrum()(frame)
        spectrum_magnitudes.append(spectrum_magnitudes_frame)
    spectrum_magnitudes = np.array(spectrum_magnitudes).flatten()
    
    frequencies, magnitudes = SpectralPeaks()(audio44k)
    hnr_score = None
    if frequencies[0]: 
        hnr_score = Inharmonicity()(frequencies, magnitudes)
    ###
    
    onset_rate_score = OnsetRate()(audio44k)
    brightness_score = SpectralCentroidTime()(audio44k)
    dynamic_complexity_score, _ = DynamicComplexity()(audio44k)
    
    mel_bands = get_mel_bands(audio44k)
    novelty_curve = NoveltyCurve()(mel_bands)
    novelty_score = np.median(np.abs(np.diff(novelty_curve)))
    
    beats_loudness, beats_loudness_band_ratio = BeatsLoudness(beats=beat_positions)(audio44k)
    beatogram = Beatogram()(beats_loudness, beats_loudness_band_ratio)
    time_signature = Meter()(beatogram)
    
    _, _, saliences, hum_starts, hum_ends = HumDetector()(audio44k)
    hum_intervals = [(hum_start, hum_end, salience) for hum_start, hum_end, salience in zip(hum_starts, hum_ends, saliences)]
        
    features = {
        'Danceability': danceability_score[0],
        'Loudness': loudness_score,
        'BPM': bpm,
        'Key': key,
        'Key Scale': scale,
        'Energy': energy_score,
        'Chords Significance': chords_significance,
        'Inharmonicity': hnr_score,
        'Timbre (MFCC Coefficients Mean)': np.mean(mfcc_coeffs),
        'Onset Rate': onset_rate_score[1],
        'Brightness': brightness_score,
        'Dynamic Complexity': dynamic_complexity_score,
        'Novelty': novelty_score,
        'Time Signature': time_signature,
        'Hum Intervals': hum_intervals
    }
    return features

In [12]:
def extract_audio_features(audio_file):
    # Load the audio file
    #audio16k = MonoLoader(filename=audio_file, sampleRate=16000)()
    audio44k = MonoLoader(filename=audio_file)()

    algorithm_features = run_essentia_algorithms(audio44k)
    spectrogram, sample_rate = convert_mp3_to_spectrogram(audio_file)

    # Merge results and return
    features = algorithm_features | {'Spectrogram': spectrogram, 'Spectrogram Sample Rate': sample_rate}
    return features

#### Main Code

In [13]:
def process_song(song, song_index, np):
    song_id = song.get('videoID')
    if not song_id:
        print(f"Error: videoID not found. Song: {song}")
        return

    # Download song
    song_path = download_song(song_id, DOWNLOAD_FOLDER)
    if not song_path:
        return

    # Extract song features
    song_features = extract_audio_features(song_path)

    for feature, value in song_features.items():
        if isinstance(value, (tuple, set, list, np.ndarray, dict)) and feature not in np.shared_songs_data.columns:
            np.shared_songs_data[feature] = np.nan
            np.shared_songs_data[feature] = np.shared_songs_data[feature].astype(object)
        np.shared_songs_data.at[song_index, feature] = value

In [14]:
def process_batch(args, np):
    song_batch, batch_num = args
    process_times = []
    song_count = 0
    
    for song_index, song in song_batch.iterrows():
        song_count += 1
        startTime = time.time()
        process_song(song, song_index, np.songs_data)
        process_time = time.time() - startTime
        process_times.append(process_time)
        print(f"Batch {batch_num}, Song {song_count}/{len(song_batch)}: {process_time:.2f} seconds. Avg extraction time: {np.mean(process_times):.2f}")
        break